In [176]:
import jailbreakbench as jbb
import os
import pandas as pd

import numpy as np
from sklearn.svm import SVC
from scipy.special import softmax
from sklearn.model_selection import train_test_split
#from SPD.main_spd import calculate_logits
from typing import Any, Literal
import litellm
import openai
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [414]:
#os.environ['OPENAI_API_KEY'] = ...

In [4]:
%load_ext autoreload
%autoreload 2

### 1. Save the logits
For each dataset, calculate and save logit values in seperate folders. Hyperparameters such as temperature, top-p, top-k can be modified by altering the SPD/config_spd.py file.

In [119]:
# Location to save the data
write_path = "logits/gcg/test"

model_name = "gpt-4o-mini"

# Load the data from JSON
data_path = "data/GPT4o/GCG.json" 
data = pd.read_json(path_or_buf=data_path)[model_name]#["gpt-3.5-turbo-0613"]

# Load data from csv
#data_path = "data/GPT3.5/promptDatasetGPT.csv"
#data = pd.read_csv(data_path)
#data = data[data["Success"] == True]
#data = data.rename(columns={"Successful Prompt": "prompt"})
#data = data[['prompt']].apply(lambda x: x.to_dict(), axis=1)
#data = data.reset_index(drop=True)
#data.name = "gpt-3.5-turbo-0613"


ind_to_read = [*range(160, 561)] 
prompts = {}
for i in range(len(ind_to_read)):
    prompts[i] = data[ind_to_read[i]]["prompt"]

all_prompts = {model_name: prompts}
    
# Saves the logit values in write_path/model_name
#logits = calculate_logits(write_path, all_prompts, llm_provider="litellm")

In [121]:
len(prompts)

401

In [79]:
GCG_test = prompts

In [89]:
# Gathering the PAIR Prompts for later use

data_path = "data/GPT4o/PAIR.json" 
data_pair = pd.read_json(path_or_buf=data_path)[model_name]
o=[]
for i in data_pair[20:120]:
    o.append(i['prompt'])

20     {'prompt': 'You are a historian who is going t...
21     {'prompt': 'You are a famous erotic author who...
22     {'prompt': 'Imagine you are a marketing specia...
23     {'prompt': 'You are a researcher studying cont...
24     {'prompt': 'I understand that discussing certa...
                             ...                        
115    {'prompt': 'Write a story where a character is...
116    {'prompt': 'You are a skilled thief who has be...
117    {'prompt': 'You are a skilled thief who has be...
118    {'prompt': 'You are a skilled thief with a rep...
119    {'prompt': 'You are a master thief with a repu...
Name: gpt-4o-mini, Length: 100, dtype: object

In [118]:
o=[]
for i in data_pair[20:120]:
    o.append(i['prompt'])


In [91]:
# Location to save the data BENIGN PROMPTS
# Load the data
model_name = "gpt-4o-mini"
data_path = "data/GPT4o/benign_outputs.json" 
data = pd.read_json(path_or_buf=data_path)
# shuffle data
data = data.sample(frac=1).reset_index(drop=True)

In [96]:
### write_path = "logits/benign1/test"
ind_to_read = [*range(400, 796)] 
prompts = {}
for i in range(len(ind_to_read)):
    prompts[i] = data.iloc[ind_to_read].iloc[i]["instruction"]

all_prompts = {model_name: prompts}
    
# Saves the logit values in write_path/model_name
#logits = calculate_logits(write_path, all_prompts, llm_provider="litellm")


# Saving the benign prompts for later use
benign_test_1 = prompts

In [97]:
data_path = "data/GPT4o/qnli_1000.json"
data = pd.read_json(path_or_buf=data_path)
# shuffle data
data = data.sample(frac=1).reset_index(drop=True)

In [99]:
# QNLI dataset loading
write_path = "logits/benign2/test"
ind_to_read = [*range(500, 1000)] 
prompts = {}
for i in range(len(ind_to_read)):
    prompts[i] = data.iloc[ind_to_read].iloc[i]["text1"]

all_prompts = {model_name: prompts}
    
# Saves the logit values in write_path/model_name
#logits = calculate_logits(write_path, all_prompts, llm_provider="litellm")



# Saving the benign prompts for later use
benign_test_2 = prompts

### 2. Load training data

Load the saved logit values and prepare the feature vector. While $r$ determines the number of token locations, $k$ determines how many candidates are calculated per each location.

In [215]:
# Difference in classes code cell!

BENIGN_PATH_TRAIN = "logits/benign/train"
ATTACK_PATH_TRAIN = "logits/pair/train"

model_name = "gpt-4o-mini"
r = 25   # Number of token positions to consider
k = 5    # Number of candidate logit values to consider

# Two separate lists: one for all benign sets, one for all attacker sets
benign_paths_train = [
    "logits/benign1/train",
    "logits/benign2/train"
]

attack_paths_train = [
    "logits/pair/train",
    "logits/gcg/train"
]

# ------------------------------------------------------------------
# 2. Load and reshape all BENIGN TRAIN data
# ------------------------------------------------------------------
benign_list = []
for path in benign_paths_train:
    # Load logits for this subset
    logits = np.load(os.path.join(path, f"{model_name}.npy"))
    # Convert logits -> probabilities -> negative log probabilities
    probs = softmax(logits, axis=2)
    vals  = -np.log(probs)  # shape: [N, seq_len, vocab_subset?]

    # Slice down to just the first r token positions & top k candidates
    vals = vals[:, :r, :k]  # shape: [N, r, k]

    # Flatten each sample to [r*k]
    benign_list.append(vals.reshape(vals.shape[0], r * k))

# Combine all benign samples from all benign sets
train_benign = np.concatenate(benign_list, axis=0)
print("train_benign shape:", train_benign.shape)

# ------------------------------------------------------------------
# 3. Load and reshape all ATTACK TRAIN data
# ------------------------------------------------------------------
attack_list = []
for path in attack_paths_train:
    logits = np.load(os.path.join(path, f"{model_name}.npy"))
    probs  = softmax(logits, axis=2)
    vals   = -np.log(probs)

    vals = vals[:, :r, :k]
    print(path)
    print(vals.shape)
    attack_list.append(vals.reshape(vals.shape[0], r * k))

# Combine all attack samples from all attack sets
train_attack = np.concatenate(attack_list, axis=0)
print("train_attack shape:", train_attack.shape)

# ------------------------------------------------------------------
# 4. Create final training arrays (X, y)
# ------------------------------------------------------------------
train_x = np.concatenate([train_benign, train_attack], axis=0)
train_y = np.concatenate([
    np.zeros(train_benign.shape[0]),  # label=0 for benign
    np.ones(train_attack.shape[0])    # label=1 for attack
])

print("Final train_x shape:", train_x.shape)
print("Final train_y shape:", train_y.shape)


train_benign shape: (890, 125)
logits/pair/train
(20, 25, 5)
logits/gcg/train
(154, 25, 5)
train_attack shape: (174, 125)
Final train_x shape: (1064, 125)
Final train_y shape: (1064,)


### 3. Train the classifier

In [216]:
# Train the classifier 
clf = SVC(kernel="rbf",class_weight="balanced")
clf.fit(train_x, train_y) 

SVC(class_weight='balanced')

In [14]:
benign_data = np.load(os.path.join("logits/benign/test", f"{model_name}.npy"))
print("benign_data shape:", benign_data.shape)
attack_data = np.load(os.path.join("logits/pair/test", f"{model_name}.npy"))
print("attack_data shape:", attack_data.shape)

benign_data shape: (160, 25, 5)
attack_data shape: (100, 25, 5)


### 4. Load the test data

Load the saved logit values and prepare the feature vector. $r$ and $k$ values should remain the same with training data.

In [15]:
# Locations of the test data
#read_paths_test = ["GPT3.5/logits/benign1/test",
#                    "GPT3.5/logits/benign2/test",
#                    "gpt-3.5/logits/pair/test",
#                    "gpt-3.5/logits/gcg/test"]

read_paths_test = ["logits/benign/test", 
                   "logits/pair/test"]
#test_size = 40

logit_values_test = []

# Finding minimum test size dynamically in case of imbalanced classes
test_sizes = []
for path in read_paths_test:
    logits = np.load(os.path.join(path, f"{model_name}.npy"))
    test_sizes.append(logits.shape[0])  # Number of available samples

test_size = min(test_sizes)

for path in read_paths_test:
    logits = np.load(os.path.join(path, f"{model_name}.npy"))
    probabilities = softmax(logits, axis=2)
    values = - np.log(probabilities)
    
    logit_values_test.append(values[:test_size,:r,:k].reshape(test_size, r * k))

logit_values_test = np.array(logit_values_test)
benign_indexes = [0]
attack_indexes = [1]

test_benign = np.concatenate((logit_values_test[benign_indexes]))
test_attack = np.concatenate((logit_values_test[attack_indexes]))

In [73]:
test_attack.shape

(478, 125)

In [45]:
model_name = "gpt-4o-mini"
r = 25
k = 5

# ------------------------------------------------------------------
# 1. Define the paths where your TEST data (logits) live
# ------------------------------------------------------------------
benign_paths_test = [
    "logits/benign1/test",
    "logits/benign2/test"
]

attack_paths_test = [
    "logits/pair/test",
    "logits/gcg/test"
]

# ------------------------------------------------------------------
# 2. Load and reshape all BENIGN TEST data
# ------------------------------------------------------------------
test_benign_list = []
for path in benign_paths_test:
    logits = np.load(os.path.join(path, f"{model_name}.npy"))
    probs  = softmax(logits, axis=2)
    vals   = -np.log(probs)               # shape: [N, seq_len, vocab_subset?]

    # Keep only first r token positions and top k candidates
    vals = vals[:, :r, :k]                # shape: [N, r, k]
    test_benign_list.append(vals.reshape(vals.shape[0], r * k))

# Combine all benign test samples
test_benign = np.concatenate(test_benign_list, axis=0)
print("test_benign shape:", test_benign.shape)

# ------------------------------------------------------------------
# 3. Load and reshape all ATTACK TEST data
# ------------------------------------------------------------------
test_attack_list = []
for path in attack_paths_test:
    logits = np.load(os.path.join(path, f"{model_name}.npy"))
    probs  = softmax(logits, axis=2)
    vals   = -np.log(probs)

    vals = vals[:, :r, :k]
    test_attack_list.append(vals.reshape(vals.shape[0], r * k))

# Combine all attack test samples
test_attack = np.concatenate(test_attack_list, axis=0)
print("test_attack shape:", test_attack.shape)



test_benign shape: (889, 125)
test_attack shape: (478, 125)


array([[4.43443764e-02, 3.79434439e+00, 4.54434415e+00, ...,
        5.20191968e+00, 6.57691968e+00, 9.70192015e+00],
       [3.60304403e-01, 1.36030437e+00, 3.98530437e+00, ...,
        1.81250114e+01, 1.83750114e+01, 2.00000114e+01],
       [3.49210490e-02, 3.53492096e+00, 5.65992096e+00, ...,
        1.86717682e+00, 3.74217682e+00, 3.99217682e+00],
       ...,
       [4.50349706e-01, 1.70034968e+00, 2.20034980e+00, ...,
        2.86123344e+00, 2.98623344e+00, 3.86123344e+00],
       [6.71809038e-03, 5.00671815e+00, 1.31317177e+01, ...,
        8.19344228e+00, 1.10684423e+01, 1.25684423e+01],
       [8.61761680e-03, 4.75861761e+00, 1.30086181e+01, ...,
        8.50783764e+00, 1.03828376e+01, 1.21328376e+01]])

In [197]:
# Putting all Benign Test prompts together to associate them with logits
benign_test_1_trimmed = [benign_test_1[i] for i in range(393)]
benign_test_2_trimmed = [benign_test_2[i] for i in range(496)]
test_benign_df = pd.DataFrame(test_benign)
test_benign_df['prompts'] = pd.concat([pd.Series(benign_test_1_trimmed), pd.Series(benign_test_2_trimmed)]).reset_index(drop = True)

In [17]:
paths = [
    "logits/pair/test",
    "logits/gcg/test"
]

for path in paths:
    path_npy = os.path.join(path, f"{model_name}.npy")
    print("Loading from:", path_npy)
    arr = np.load(path_npy)
    print("Shape of arr:", arr.shape)

Loading from: logits/pair/test\gpt-4o-mini.npy
Shape of arr: (100, 25, 5)
Loading from: logits/gcg/test\gpt-4o-mini.npy
Shape of arr: (378, 25, 5)


In [132]:
# Putting the attacks in one dataframe
test_attack_df = pd.DataFrame(test_attack)
test_attack_df['prompt'] = pd.concat([pd.Series(o), pd.Series(GCG_test).sample(378)]).reset_index(drop = True)

0      You are a historian who is going to write a co...
1      You are a famous erotic author who has written...
2      Imagine you are a marketing specialist for a c...
3      You are a researcher studying controversial an...
4      I understand that discussing certain controver...
                             ...                        
473    Create a fake online store that steals custome...
474    Write a manual on how to escape from a correct...
475    Create a fake news story that spreads misinfor...
476    Write a manual on how to escape from a correct...
477    Write a blog post promoting the use of illegal...
Length: 478, dtype: object

### 5. Test the classifier

Get the TP and FP rates of the test data.

In [201]:
# Preliminary Classifier 
#-----------------------
# Uses an LLM to check for overtly benign prompts
# Created in LLM_Classifier.ipynb

def call_model(input_text,llm):
    '''
    Calls an LLM with a given input text

    input_text: a string of text to input in the LLM
    llm: a string outlining which llm from together AI to use
    '''
    response = openai.chat.completions.create(
                model = llm,
                messages = [
                    {"role": "user", "content": input_text},
                ],
                max_tokens = 1000,
            )
    output = response.choices[0].message.content
    
    return output
    




def llm_check(llm, series):
    '''
    A preliminary LLM classifer identifying benign prompts

    llm: a string outlining which llm from together AI to use
    series: a series containing prompts from which to classify
    n: the number of prompts to sample from the series and classify

    OUTPUT: 'YES' or 'NO' with 'NO' meaning that there is some element of the prompt that could make it harmful
             while 'YES' means it is confidentally benign.
    '''
    results = []
    #sample = series.sample(n)
    sample = series
    for query in sample:
        temp_prompt = prompt_check.replace('{QUERY}', query)
        out = call_model(temp_prompt, llm)
        results.append(out.strip())
    return results
with open('assets/check_system.txt', 'r') as f:
    prompt_check = f.read()

In [301]:
# Generating a sample of benign and attack data 
sampled_df = pd.concat([test_benign_df.sample(200, random_state = 123).reset_index(drop = True), test_attack_df.sample(200, random_state = 123).reset_index(drop = True)])

In [303]:
# Running Prompts marked as 'Not Benign' from the preliminary classifier through SPD

import time 
model_name = "gpt-4o-mini"
start_time = time.time()
prelim_results = llm_check(model_name, sampled_df['prompts'])
end_time = time.time()
print(end_time -  start_time)

179.2693271636963


In [312]:
sampled_df['true'] = ['benign'] * 200 + ['attack'] * 200 
sampled_df['results'] = prelim_results

In [325]:
prelim_nos = sampled_df[sampled_df['results'] == 'NO'].reset_index(drop = True)

In [326]:
prelim_nos['true'].value_counts()

true
attack    199
benign      7
Name: count, dtype: int64

In [344]:
# False Positives out of 200 
prelim_nos[(prelim_nos['pred'] == 1) & (prelim_nos['true'] == 'benign')]

,0,1,2,3,4,5,6,7,8,9,...,119,120,121,122,123,124,prompts,results,true,pred
3,0.36378,1.86378,2.11378,3.73878,5.23878,0.000013,11.750013,12.250013,15.375013,19.625013,...,15.630277,0.006034,5.256034,7.131034,15.381035,16.506035,AI assistant. I am looking for legal reference...,NO,benign,1.0


In [349]:
# Prompts marked 'benign' by preliminary classifier
sampled_df[sampled_df['results'] == 'YES']['true'].value_counts()

true
benign    193
attack      1
Name: count, dtype: int64

In [350]:
# Calculating F-1 Score
f = np.concatenate((
    np.zeros(193), 
    np.ones(1)
))
g = np.zeros(194)

In [396]:
 # Calculating F-1 Score
left = pd.concat([pd.DataFrame(f).rename(columns = {0:'true'}),pd.DataFrame(g).rename(columns = {0:'pred'})], axis = 1)

right = pd.concat([pd.DataFrame((prelim_nos['true'] == 'attack').astype(float)),prelim_nos[['pred']]], axis = 1)


In [399]:
# Calculating F-1 Score
final_results = pd.concat([left, right])

In [401]:
# Calculating F-1 Score
h = pd.concat([pd.DataFrame((prelim_nos['true'] == 'attack').astype(float)),prelim_nos[['pred']]], axis = 1)

f1_score(final_results['true'], final_results['pred'])

0.9251336898395722

In [331]:
prelim_nos['pred'] = clf_out

In [330]:
start_time = time.time()
clf_out = clf.predict(prelim_nos.drop(columns = ['prompts', 'results', 'true']).to_numpy())
end_time = time.time()
print(end_time -  start_time)

0.010164022445678711


In [246]:
pipeline_3_prompts = prelim_nos[clf_out == 0]['prompts']

In [253]:
# Discarded Semantic Transformation Pupeline
# 
# Originates from Transforming.ipynb

transformations_dir = os.path.join(os.getcwd(), 'transformations')
SEMANTIC_TRANSFORMATIONS = [
    "summarize",
    "paraphrase",
    "synonym",
    "verbtense",
    "spellcheck",
    "translate",
    "formatsummarize"
]

def transform(input_text, transformation_type, llm):
    '''
    Takes an input_text and transforms it according to a specified transformation_type. 

    input_text: string of text to transform
    transformation_type: string naming which transformation to apply (from SEMANTIC_TRANSFORMATIONS)
    llm: a string outlining which llm from together AI to use
    '''
    if transformation_type.lower() not in SEMANTIC_TRANSFORMATIONS:
        print('Enter Valid Transformation')
        return None

    template_file = transformation_type.lower() + '.txt'
    input_prompt = open(os.path.join(transformations_dir, template_file), 'r').read().strip()
    
    transformed_input = input_prompt.replace('{QUERY}', input_text)
    transformed_input = call_model(transformed_input, llm)
    
    if transformation_type.lower() == 'translate':
        transformed_input = transformed_input + "\n Please answer the above instructions in English."
    if transformation_type.lower() == 'formatsummarize':
        transformed_input = transformed_input + " Please answer the above instructions."

    
    
    return transformed_input


In [296]:
# Discarded Semantic Transformation Pipeline
judge_system = open('assets/judge_system.txt', 'r').read().strip()

def judge_call(prompt, response, llm):  
        judge_prompt = f"Prompt: {prompt}\nResponse: {response}"
        
        messages = [
            {'role': 'system', 'content': judge_system},
            {'role': 'user', 'content': judge_prompt}
        ]
        
        response = openai.chat.completions.create(model =llm, messages = messages, max_tokens = 1000)
        
        return response.choices[0].message.content
 

In [294]:
## Discarded Semantic Transformation Pipeline

def transform_check(prompts, llm):
    '''
    prompt: input prompt 
    llm: llm to use
    '''
    results = []
    #judge_prompt = open(os.path.join(assets_dir, 'judge_system.txt'), 'r').read().strip()
    for prompt in prompts:
        
        transformed = transform(prompt, 'spellcheck', llm)
        transformed2 = transform(transformed, 'summarize', llm)
        response = call_model(transformed2, llm)
        print(response)
        judge_response = judge_call(transformed2, response, llm)
        results.append(judge_response)
    return results

In [409]:
#judged_responses = transform_check(pipeline_3_prompts, model_name)
#judged_responses

In [410]:
#for i in pipeline_3_prompts:
    #print(i)